<h1>Grafo delle affiliations relativo al dataset degli articoli di informatica</h1>
<p>
    <ul>
        <li>Nodi: istituzioni/università</li>
        <li>Archi: collaborazione tra autori delle due istituzioni</li>
    </ul>
</p>

In [17]:
from matplotlib.font_manager import font_scalings
from pyvis.network import Network
import networkx as nx
import itertools
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
import re
from collections import Counter

<h2>Carica il dataset</h2>

In [18]:
# Carica il dataset completo
df_affiliations = pd.read_csv("../data/informatica.csv")

<h3>Filtri per estrarre solamente le istituzioni che collaborano almeno in 10 articoli</h3>

In [19]:
# Funzione per estrarre istituzioni da una cella
def extract_institutions(cell):
    if pd.isna(cell):
        return []
    parts = [x.strip() for x in str(cell).split(";")]
    # elimina stringhe vuote
    return [p for p in parts if p]

In [20]:
# Conta le istituzioni
counter = Counter()

for affiliations in df_affiliations["Affiliations"]:
    inst_list = extract_institutions(affiliations)
    counter.update(inst_list)

# -----------------------------
# 2) Filtra istituzioni con almeno 10 occorrenze
# -----------------------------
min_occ = 10
istituzioni_valid = {inst for inst, c in counter.items() if c >= min_occ}

print(f"Istituzioni presenti almeno {min_occ} volte: {len(istituzioni_valid)}")

Istituzioni presenti almeno 10 volte: 161


<h3>Creazione del dataframe delle istituzioni con relativa città e nazione</h3>

In [24]:
def is_valid_city(token):
    """
    Ritorna True se il token può essere considerato un nome di città
    e non una sigla di provincia (UD, MI, RM, ecc.).
    """
    token_clean = token.strip()

    # escludi stringhe troppo corte
    if len(token_clean) <= 2:
        return False

    # escludi codici di provincia in maiuscolo
    if token_clean.isupper():
        return False

    # escludi numeri
    if token_clean.isnumeric():
        return False

    # escludi pattern tipo "UD", "MI", "RM", "TS"
    if re.fullmatch(r"[A-Z]{2,3}", token_clean):
        return False

    return True

def parse_institution(inst):
    """
    Ritorna (nome, città, nazione) usando euristica migliorata:
    - la nazione è sempre l'ultimo elemento
    - la città è l'ultimo elemento valido prima della nazione
    """
    parts = [p.strip() for p in inst.split(",")]

    # caso con 1 elemento → solo nome
    if len(parts) == 1:
        return pd.Series([parts[0], None, None])

    # nazione = ultimo elemento
    nazione = parts[-1]

    # cerca la città tra gli elementi prima della nazione
    candidate_city = None
    for elem in reversed(parts[:-1]):
        if is_valid_city(elem):
            candidate_city = elem
            break

    # se nessuna città valida trovata
    if candidate_city is None:
        nome = ", ".join(parts[:-1])
        return pd.Series([nome, None, nazione])

    # nome = tutto ciò che viene prima della città
    city_index = parts.index(candidate_city)
    nome = ", ".join(parts[:city_index])

    return pd.Series([nome, candidate_city, nazione])

# Costruzione DataFrame
df_istituzioni = pd.DataFrame(list(istituzioni_valid), columns=["raw"])
df_istituzioni[["nome", "città", "nazione"]] = df_istituzioni["raw"].apply(parse_institution)
df_istituzioni = df_istituzioni[["nome", "città", "nazione"]]

In [23]:
df_istituzioni

,nome,città,nazione
0,Abdus Salam International Centre for Theoretic...,Trieste,Italy
1,"Department of Computer Science, Università di ...",Pisa,Italy
2,Università degli Studi di Siena,Siena,Italy
3,"Università degli Studi di Udine, Dipartimento ...",Udine,Italy
4,INRIA Institut National de Recherche en Inform...,Ile-de-France,France
...,...,...,...
156,Universidad de Murcia,Murcia,Spain
157,Università degli Studi di Napoli Federico II,Naples,Italy
158,"Infineon Technologies Austria AG, Villach",Carinthia,Austria
159,"Communication, Università degli Studi di Udine",Udine,Italy


<h2>Grafo</h2>

In [29]:
G_filt = nx.Graph()

for affiliations in df_affiliations["Affiliations"]:
    inst_list = extract_institutions(affiliations)

    # Applica il filtro
    inst_list = [inst for inst in inst_list if inst in istituzioni_valid]

    # Se rimane meno di 2 istituzioni, non può formarsi un arco
    if len(inst_list) < 2:
        continue

    # aggiungi nodi
    for inst in inst_list:
        if inst not in G_filt:
            G_filt.add_node(inst)

    # crea archi
    for a, b in itertools.combinations(inst_list, 2):
        if G_filt.has_edge(a, b):
            G_filt[a][b]["weight"] += 1
        else:
            G_filt.add_edge(a, b, weight=1)

print("Grafo filtrato creato!")
print(f"Nodi: {G_filt.number_of_nodes()}")
print(f"Archi: {G_filt.number_of_edges()}")

Grafo filtrato creato!
Nodi: 161
Archi: 1988


<h3>Visualizzazione del grafo</h3>

In [26]:
net = Network(height="900px", width="100%", filter_menu=True, bgcolor="white", font_color="black")
net.barnes_hut()

# Aggiungi nodi
for node in G_filt.nodes():
    net.add_node(
        node,
        label=node,
        title=node,
        size=12,
        color="#4A90E2"
    )

# Aggiungi archi
for u, v, data in G_filt.edges(data=True):
    peso = data.get("weight", 1)
    net.add_edge(
        u, v,
        value=peso,
        title=f"Collaborazioni: {peso}"
    )

output_path = "../html/grafo_affiliations/grafo_istituzioni_filtrate.html"
net.write_html(output_path)
print("File generato:", output_path)
net.save_graph(output_path)

File generato: ../html/grafo_affiliations/grafo_istituzioni_filtrate.html


<h1>Analisi del grafo</h1>

<h2>Degree</h2>

In [32]:
degree_dict = dict(G_filt.degree())
degree_centrality = nx.degree_centrality(G_filt)

<h2>Betweenness</h2>

In [33]:
betweenness = nx.betweenness_centrality(G_filt, normalized=True, weight='weight')
# betw è un dict: {nodo: valore}

<h2>Closeness</h2>

In [35]:
# --- Verifica se il grafo ha pesi sugli archi ---
has_weight = any("weight" in data for _, _, data in G_filt.edges(data=True))

# --- Calcolo closeness pesata (se ci sono pesi) ---
if has_weight:
    # NetworkX interpreta 'distance=' come distanza da minimizzare.
    # Se i pesi rappresentano distanza → usa direttamente distance='weight'
    # Se i pesi rappresentano forza → usa la conversione 1/weight
    for u, v, data in G_filt.edges(data=True):
        w = data['weight']
        data['distance'] = 1 / w if w != 0 else float("inf")

    closeness = nx.closeness_centrality(G_filt, distance='distance')
else:
    closeness = None

In [42]:
df_analisi_grafo = pd.DataFrame({
    "degree": degree_dict,
    "degree_centrality": degree_centrality,
    "betweenness": betweenness,
    "closeness": closeness
}).rename_axis("nome").reset_index()

In [43]:
df_analisi_grafo

,nome,degree,degree_centrality,betweenness,closeness
0,"Department of Mathematics, Università degli St...",63,0.39375,0.118465,3.402359
1,"Department of Management and Engineering, Univ...",10,0.06250,0.012673,1.638833
2,"Università di Trento, Trento, TN, Italy",57,0.35625,0.018499,3.458128
3,"Università degli Studi di Udine, Udine, UD, Italy",122,0.76250,0.079303,4.029802
4,"Università degli Studi di Udine, Dipartimento ...",7,0.04375,0.013525,1.873445
...,...,...,...,...,...
156,"Courant Institute of Mathematical Sciences, Ne...",8,0.05000,0.005415,2.428648
157,"IEEE, New York, NY, United States",17,0.10625,0.005393,2.880742
158,"Dipartimento Ingegneria Elettrica, Università ...",1,0.00625,0.000000,3.021480
159,Department of Biophysical and Electronic Engin...,6,0.03750,0.000000,2.177995
